# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [39]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.train.automl import AutoMLConfig

from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core import Run

import os
import pandas as pd

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-exp'

experiment = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: udacity_ws
Azure region: eastus
Subscription id: 051560d6-9344-4907-a9c5-057add5cf030
Resource group: udacity_rg


In [4]:
cluster_name = "aml-compute1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {cluster_name}")
except ComputeTargetException:
    print(f"Creating new compute target: {cluster_name}")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_DS3_v2",
        min_nodes=0,
        max_nodes=4
    )

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found existing compute target: aml-compute1


In [6]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
from azureml.core import Dataset
import os

dataset_name = 'denver-cpi'

try : 

    ds = Dataset.get_by_name(ws, dataset_name)
    print(f"Dataset '{dataset_name}' found and loaded.")
    
except Exception as e:
    path = kagglehub.dataset_download("bls/denver-cpi")
    path = os.path.join(path, "Consumer_Price_Index_in_Denver.csv")
    datastore = ws.get_default_datastore()
    df = pd.read_csv(path)

    ds = Dataset.Tabular.register_pandas_dataframe(
            dataframe = df, 
            name=dataset_name,
            description='CPI denver dataset',
            target=datastore
        )

    # profile_run_config = DatasetProfileRunConfig(compute_target=compute_target)
    # data_profile = DataProfile(profile_run_config)
    # profile_result = data_profile.profile(ds)

Dataset 'denver-cpi' found and loaded.


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [26]:
early_termination_policy = BanditPolicy(evaluation_interval = 2, slack_factor= 0.1)

param_sampling = RandomParameterSampling( {
        "n_estimators": choice(25, 50, 75, 100),
        "max_depth": choice(2, 3, 4),
        "min_samples_split": choice(3, 4, 5),
        "learning_rate": choice(0.1, 0.01)
    }
)

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='sk_dep.yaml')

src = ScriptRunConfig(
    source_directory='.',
    script='train.py',  
    environment=sklearn_env, 
    compute_target = compute_target
)

hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name='normalized_root_mean_squared_error',
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=30,
    max_concurrent_runs=4,
    max_duration_minutes = 30
    )

In [27]:
hyperdrive_run = experiment.submit(hyperdrive_config)

In [33]:
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-exp,HD_854e156b-ca56-4be2-abdf-f3f1f20da835,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
# from azureml.widgets import RunDetails
# RunDetails(hyperdrive_run).show()

In [34]:
for run in experiment.get_runs():
    print(f"Run ID: {run.id} | Status: {run.status}")

Run ID: HD_854e156b-ca56-4be2-abdf-f3f1f20da835 | Status: Completed
Run ID: HD_f714d873-1611-4c2f-8c3c-8968a470887d | Status: Failed
Run ID: HD_e16e2ab0-055f-44f3-8d92-58f02a71c967 | Status: Failed
Run ID: HD_b9983925-60f4-4148-842d-191405825f61 | Status: Failed
Run ID: HD_e1d7a60e-c411-44d2-8a09-ac0637c3fd41 | Status: Failed
Run ID: 058072b4-d40c-4714-b6be-5f179f0467f9 | Status: Running


## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [41]:
all_metrics = hyperdrive_run.get_metrics()

best_run_id = min(
    all_metrics.items(),
    key=lambda x: x[1].get('MSE', float('inf'))
)[0]

print("Best run ID:", best_run_id)

best_run = Run(experiment=hyperdrive_run.experiment, run_id=best_run_id)
best_run

Best run ID: HD_854e156b-ca56-4be2-abdf-f3f1f20da835_24


Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-exp,HD_854e156b-ca56-4be2-abdf-f3f1f20da835_24,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [42]:
metrics = best_run.get_metrics()
for key, value in metrics.items():
    print(f"{key}: {value}")

max_depth:: 4
min_samples_split:: 4
n_estimators:: 100
learning_rate:: 0.1
MSE: 2.2483377766399393


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [45]:
model = best_run.register_model(model_name='hyperdrive_model', model_path='.')
model

Model(workspace=Workspace.create(name='udacity_ws', subscription_id='051560d6-9344-4907-a9c5-057add5cf030', resource_group='udacity_rg'), name=hyperdrive_model, id=hyperdrive_model:1, version=1, tags={}, properties={})

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

